In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['WANDB_MODE'] = 'offline'

In [47]:
from modules import ImageEncoder

In [4]:
from nnunetv2.run.run_training import get_trainer_from_args
import torch

trainer = get_trainer_from_args(
    dataset_name_or_id='302',
    configuration='3d_32x512x512_b2',
    fold=0,
    trainer_name='nnUNetTrainer',
    plans_identifier='nnUNetPlans',
    device=torch.device('cpu'))
trainer.initialize()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Using device: cpu



#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-07-17 14:25:51.559287: Using torch.compile...


/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [6]:
encoded_image = trainer.network.encoder(torch.rand(2, 1, 32, 512, 512))
len(encoded_image)

6

In [7]:
for i, feature in enumerate(encoded_image):
    print(f'feature [{i}]: {feature.shape}')

feature [0]: torch.Size([2, 32, 32, 512, 512])
feature [1]: torch.Size([2, 64, 16, 256, 256])
feature [2]: torch.Size([2, 128, 8, 128, 128])
feature [3]: torch.Size([2, 256, 4, 64, 64])
feature [4]: torch.Size([2, 320, 2, 32, 32])
feature [5]: torch.Size([2, 320, 2, 16, 16])


In [28]:
# torch.nn.functional.
from timm.models.layers import create_classifier

global_pool, classifier = create_classifier(
    num_features=len(encoded_image),
    num_classes=0,
    pool_type='avg',
)

In [39]:
pool = torch.nn.Sequential(
    torch.nn.AdaptiveAvgPool3d(1),
    torch.nn.Flatten(),
)

In [40]:
for i, feature in enumerate(encoded_image):
    print(f'Pooled feature [{i}]: {pool(feature).shape}')

Pooled feature [0]: torch.Size([2, 32])
Pooled feature [1]: torch.Size([2, 64])
Pooled feature [2]: torch.Size([2, 128])
Pooled feature [3]: torch.Size([2, 256])
Pooled feature [4]: torch.Size([2, 320])
Pooled feature [5]: torch.Size([2, 320])


In [43]:
pooled_features = [pool(feature) for feature in encoded_image]
pooled_features = torch.cat(pooled_features, axis=1)
pooled_features.shape

torch.Size([2, 1120])

In [48]:
encoder = ImageEncoder()

Using device: cpu



#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-07-17 15:07:27.477615: Using torch.compile...


/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [50]:
encoder(torch.rand(2, 1, 32, 512, 512)).shape

torch.Size([2, 1120])